In [ ]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from train_trl import TrainerWrapper, RLConfig, LLAMA_3_2_1B, SMOL_LM_135M

In [43]:
cfg = RLConfig(
    single_process_mode=True,
    max_samples=100,
    model_id=SMOL_LM_135M,
)
wrapper = TrainerWrapper(cfg)
wrapper.init_model()
wrapper.init_data_module(False)

2024-11-27 16:44:55.281 | INFO     | train_trl:init_model:45 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct
2024-11-27 16:44:58.189 | INFO     | dataset.squad:setup:220 - Loading dataset for stage fit
2024-11-27 16:44:59.236 | INFO     | dataset.squad:setup:224 - Loaded dataset with 60917 samples
2024-11-27 16:44:59.275 | INFO     | dataset.squad:setup:232 - Processing dataset for stage fit, workers: 1, cache dir dataset_caches/ultrafeedback
Map: 100%|██████████| 10/10 [00:00<00:00, 527.61 examples/s]


In [ ]:
wrapper.tokenizer.eos_token, wrapper.tokenizer.pad_token

In [46]:
wrapper.init_trainer()

2024-11-27 16:45:06.483 | INFO     | train_trl:init_trainer:87 - LoRA already loaded
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
2024-11-27 16:45:06.836 | INFO     | train_trl:init_trainer:122 - Initializing DPOTrainer


In [50]:
wrapper.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49153, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=576, out_features=576, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=576, out_features=256, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=256, out_features=576, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=576, out_features=192, bias=False)
            (lora_dropout): Module

In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [ ]:
wrapper.compute_loss_metrics()
# need to use no_grad or get OOM
# first_batch = next(iter(wrapper.trainer.get_train_dataloader()))
# print(wrapper.tokenizer.decode(first_batch['prompt_input_ids'][0]))
# with torch.no_grad():
#     for batch in wrapper.trainer.get_train_dataloader():
#         loss, out = wrapper.trainer.compute_loss(wrapper.model, batch, True)
#         print(loss)
#         display(out)

In [ ]:
# plot distributiuon of losses
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

STR_COLS = ['prompt', 'chosen', 'rejected']

out_df = pd.DataFrame(outputs)

for col in out_df.columns:
    if col in STR_COLS:
        continue
    print(out_df[col].describe())
    plt.hist(out_df[col], bins=1000)
    plt.title(col)
    plt.show()